# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [1]:
import pyorient

In [2]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [3]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [4]:
client.db_create(name='gods', type=pyorient.DB_TYPE_GRAPH, storage=pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [5]:
cluster_id = client.db_open('gods', "root", "root" )

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [6]:
cluster_id = client.command( "create vertex set name = 'Zeus', symbol = 'thunder'" )

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [7]:
vertices = client.command( "SELECT * FROM V" )
for v in vertices:
    print(v.name)

Zeus


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [8]:
gods = [('Héra', 'tiara'), ('Poséidon', 'trident'),
        ('Athena', 'helmet'), ('Arès', 'weapons')]
for name, symbol in gods:
    client.command(f"create vertex set name = '{name}', symbol = '{symbol}'" )

**Q:** Display all vertices with name = `Arès`

In [9]:
vertices = client.command( "SELECT * FROM V WHERE name = 'Arès'" )
for v in vertices:
    print(v.name, v)

Arès {'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [10]:
zeus_to_poseidon = client.command("CREATE EDGE FROM (SELECT * FROM V WHERE name = 'Zeus') TO (SELECT * FROM V WHERE name = 'Poséidon') set kind = 'sibling'")

**Q:** Redisplay all vertices, discuss.

In [11]:
vertices = client.command( "SELECT * FROM V" )
for v in vertices:
    print(v.name)

Zeus
Héra
Poséidon
Athena
Arès


**Q:** Display all edges. They are contained in the class `E`

In [12]:
edges = client.command( "SELECT * FROM E" )
for e in edges:
    print(e.out)

#9:0


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [13]:
_edges = [('Zeus', 'Héra', 'sibling'), ('Zeus', 'Arès', 'father'),
            ('Zeus', 'Athena', 'father'), ('Héra', 'Arès', 'mother'),
            ('Héra', 'Zeus', 'sibling'), ('Poséidon', 'Zeus', 'sibling')]
for _from, _to, _kind in _edges:
    client.command(f"CREATE EDGE FROM (SELECT FROM V WHERE name = '{_from}') TO (SELECT FROM V WHERE name = '{_to}') SET kind = '{_kind}'")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [14]:
edges = client.command( "SELECT expand(out().name) FROM V WHERE name = 'Zeus'" )
for e in edges:
    print(e)

{{'value': 'Poséidon'},'version':0,'rid':'#-1:-1'}
{{'value': 'Héra'},'version':0,'rid':'#-1:-1'}
{{'value': 'Arès'},'version':0,'rid':'#-1:-1'}
{{'value': 'Athena'},'version':0,'rid':'#-1:-1'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [15]:
edges = client.command( "SELECT expand(in) FROM E WHERE kind = 'father'" )
for e in edges:
    print(e)

{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x1048b8810>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x10492ca10>},'version':2,'rid':'#12:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [16]:
V = client.command( "SELECT * FROM V WHERE @rid in (SELECT out FROM E where kind = 'mother')" )# 
for e in V:
    print(e.name)

Héra


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [17]:
# Todo
edges = client.command( "SELECT expand(in) FROM E WHERE kind = 'sibling'" )
for e in edges:
    print(e.name)

Poséidon
Héra
Zeus
Zeus


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [18]:
client.db_create('eCommerce', pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)
client.db_open('eCommerce', "root", ROOT_PASSWORD)

In [19]:
cluster_id = client.command( "create class products extends V" )

In [20]:
cluster_id = client.command( "create class clients extends V" )

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [21]:
cluster_id = client.command("create property products.name String")
products = ['spaghetti', 'bolognese sauce', 'cheese', 'apple']
for p in products:
    client.command(f"insert into products (name) values('{p}')")

**Q:** Create the following people: `peter`, `meredith`.

In [22]:
cluster_id = client.command("create property clients.nickname String")
clients = ['peter', 'meredith']
for name in clients:
    client.command(f"insert into clients (nickname) values('{name}')")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016

In [23]:
purchases = [('peter', ['spaghetti', 'cheese'], '10/01/2016'),
             ('meredith', ['cheese', 'apple', 'bolognese sauce'], '22/01/2016'),
             ('peter', ['spaghetti', 'bolognese sauce'], '27/01/2016')]
for c, p, date in purchases:
    client.command(f"CREATE EDGE FROM (SELECT * FROM clients WHERE nickname = '{c}') TO (SELECT * FROM products WHERE name in {str(p)}) set date = '{date}'")

**Q:** Who bought Bolognese sauce?

In [24]:
edges = client.command("SELECT expand(in_) FROM products WHERE name = 'bolognese sauce'")
for e in edges:
    client_ = client.command(f"SELECT * FROM clients where @rid = '{e.out}'")
    print(client_[0].nickname.capitalize())

Meredith
Peter


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [25]:
edges = client.command("SELECT expand(in().out()) FROM products WHERE name = 'bolognese sauce'")
products_ = []
for e in edges:
    if(e.name != 'bolognese sauce'):
        products_.append(e.name)
print(f"Products purchased with 'bolognese sauce': {set(products_)}")

Products purchased with 'bolognese sauce': {'spaghetti', 'cheese', 'apple'}


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.